<a href="https://colab.research.google.com/github/DmtrRm/IAAC2024_tutorials/blob/main/kaggle_complete2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LIBRARIES IMPORT**

In [215]:
import calendar
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler

**FUNCTIONS THAT WILL BE USED**

In [216]:
def encode_binary_categorical(data_set):
  binary_Lbls=[]
  categor_Lbls=[]

  for c in data_set.columns:

    if len(data_set[c].unique())<=3 and any(x=="Yes" for x in data_set[c].unique()):
      le=LabelEncoder()
      res =le.fit_transform(data_set[c])
      data_set[c]=1-res
      binary_Lbls.append(c)

    elif data_set[c].dtype=="object" and c != "day":
      le=LabelEncoder()
      res =le.fit_transform(data_set[c])
      data_set[c]=res
      categor_Lbls.append(c)

  return categor_Lbls

In [217]:
def find_pair_list(corralation,cor_value):


  pair_list=[]
  x=corralation.shape[0]
  y=corralation.shape[1]

  for i in range(0,x):
    for j in range(0,y):
      if (corralation.iloc[i,j]>=cor_value or corralation.iloc[i,j]<=(-cor_value)) and corralation.iloc[i,j]!=1 :
        pair_list=pair_list+[(sorted([corralation.columns[i], corralation.columns[j]]))]

  groups = []

  for pair in pair_list:
    added_to_group = False
    for group in groups:
      if any(variable in group for variable in pair):
        group.extend(pair)
        group = list(set(group))
        added_to_group = True
        break
    if not added_to_group:
      groups.append(pair)

  result = [list(set(group)) for group in groups]
  return result

In [218]:
def data_cleaning(i):


## dropping same value clm
  for c in i.columns:
   if len(i[c].unique())==1:
    i=i.drop([c], axis=1)

## encoding
  cLbls_i=encode_binary_categorical(i)

## droping dublicates
  result_i=find_pair_list(i.corr(),0.9)
  print(result_i)
  columns_to_drop = sum([x[0:-1] for x in result_i],[])
  i=i.drop(columns=columns_to_drop)

## filling the NaN
  cols_to_drop = []
  for c in i.columns:
    if len(i[c].isna().unique())>1:
      cols_to_drop.append(c)
  print(cols_to_drop)

  if not cols_to_drop==False:
    for n in cols_to_drop:
      X = i.loc[i[n].isna()==False].drop(columns=cols_to_drop)
      Y = i.loc[i[n].isna()==False][n]
      missing_X = i.loc[i[n].isna()].drop(columns=cols_to_drop)
      tree = DecisionTreeRegressor()
      tree.fit(X, Y)

      zipped = [(x, i) for i,x in sorted(zip(tree.feature_importances_, X), reverse=True)]

      i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)

  return i

In [219]:
def split_time_data(df:pd.DataFrame):
    _df = df.sort_values(by=["day"])
    split_percent = 0.8

    X = _df.drop(columns=[target]).loc[_df[target].isna()==False]
    Y = _df[target].loc[_df[target].isna()==False]

    X_train = X[:int(X.shape[0] * split_percent)].drop(columns=["day"])
    X_valid = X[int(X.shape[0] * split_percent):].drop(columns=["day"])
    Y_train = Y[:int(X.shape[0] * split_percent)]
    Y_valid = Y[int(X.shape[0] * split_percent):]

    return X_train, X_valid, Y_train, Y_valid

In [220]:
def make_shifts(df, shifts=3):
    _df_sorted = df.sort_values(by=["building_id", "meter", "day"])
    _lags = pd.DataFrame({"lag_{}".format(i): _df_sorted[target].shift(i+1) for i in range(shifts)})
    _df_sorted_lags = pd.concat([_df_sorted, _lags], axis=1)

    """
    Since we have changed the order of the rows, all the buildings except for the first one will get the input from another building on the first day.
    Let's fix it and assign a missing value to the first day lag:
    """
    for i in range(shifts):
        for j in range(0, i+1):
            _df_sorted_lags.loc[_df_sorted_lags["day"]==_df_sorted_lags["day"]- timedelta(days=int(j)), "lag_{}".format(i)] = 0

    _df_sorted_lags = _df_sorted_lags.sort_index(axis=0)
    return _df_sorted_lags

In [221]:
def return_lags(day, bid, meter, shifts):
    """
    Function to generate lags from the existing values and predictions.

    day = total_df["day"].iloc[122]

    Our original format was {'lag_{}'.format(ii): Y_train.shift(ii) for ii in shifts}
    """
    # conditions to filter the df on
    res = {}
    for shift in range(shifts):
        try:
            res['lag_{}'.format(shift)] = (total_df.loc[
                (total_df["building_id"]==bid) &
                (total_df["day"]==day - timedelta(days=int(shift+1))) &
                (total_df["meter"]==meter)]["meter_reading"].values[0])
        except Exception as e:
            res['lag_{}'.format(shift)] = 0
    return res

In [222]:
def return_datedFrame(dataFrame):
  dataFrame["day"] = dataFrame["day"].apply(lambda s: datetime.strptime(s, "%Y-%m-%d"))
  dataFrame["year"] = dataFrame["day"].apply(lambda s: s.year)
  dataFrame["month"] = dataFrame["day"].apply(lambda s: s.month)
  dataFrame["day_number"] = dataFrame["day"].apply(lambda s: s.day)
  dataFrame["weekday"] = dataFrame["day"].apply(lambda s: s.weekday() in [5,6])
  return dataFrame

**DATA UPLOADING**

In [238]:
metadata = pd.read_csv("/content/DATA/metadata.csv", index_col=0)
meters = pd.read_csv("/content/DATA/meters.csv", index_col=0)
test = pd.read_csv("/content/DATA/test.csv", index_col=0)
train = pd.read_csv("/content/DATA/train.csv", index_col=0)
weather = pd.read_csv("/content/DATA/weather.csv", index_col=0)

In [239]:
metadata = metadata.drop(columns="site_id_kaggle")

In [240]:
weather[['day', 'time']] = weather['timestamp'].str.split(' ', expand=True)
weather = weather.drop(columns='timestamp')

In [241]:
##dropping day
wth_day = weather["day"]
weather=weather.drop(columns="day")

train_day = train["day"]
train=train.drop(columns="day")

**DATA CLEANING**

In [242]:
metadata = data_cleaning(metadata)
train = data_cleaning(train)
weather = data_cleaning(weather)

[['sqft', 'sqm'], ['timezone', 'lng', 'heatingtype', 'lat']]
['yearbuilt', 'numberoffloors', 'occupants']
[]
[]


<ipython-input-218-7fc57c5a7bb4>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-input-218-7fc57c5a7bb4>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-input-218-7fc57c5a7bb4>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)


[]
['airTemperature', 'cloudCoverage', 'dewTemperature', 'precipDepth6HR', 'seaLvlPressure', 'windDirection', 'windSpeed']


<ipython-input-218-7fc57c5a7bb4>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-input-218-7fc57c5a7bb4>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-input-218-7fc57c5a7bb4>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-i

In [243]:
## inserting day
weather.insert(loc=1, column='day', value=wth_day)
train.insert(loc=1, column='day', value=train_day)



In [244]:
weather = return_datedFrame(weather)
train = return_datedFrame(train)
test = return_datedFrame(test)

In [245]:
site_ids = weather["site_id"].unique()
dfs = []

for site_id in site_ids:
    w_stats = weather.loc[weather["site_id"]==site_id].set_index("day").resample('D')[['airTemperature',
                                                                        "windSpeed"]].agg(['min', 'max', "mean"]).to_records()
    w_stats_df = pd.DataFrame(w_stats)
    w_stats_df["site_id"] = site_id
    dfs.append(w_stats_df)

w_df = pd.concat(dfs, axis=0)  # concatenating the resulting dataframes to have one

print("Shape:", w_df.shape)
w_df.head()

Shape: (3495, 8)


,day,"('airTemperature', 'min')","('airTemperature', 'max')","('airTemperature', 'mean')","('windSpeed', 'min')","('windSpeed', 'max')","('windSpeed', 'mean')",site_id
0,2016-02-01,9.1,14.2,11.782609,2.9,10.4,6.343478,3
1,2016-02-02,6.0,12.0,9.700000,2.7,13.0,8.266667,3
2,2016-02-03,8.1,13.9,12.004167,5.8,13.1,9.887500,3
3,2016-02-04,2.4,10.4,6.925000,3.1,10.7,6.433333,3
4,2016-02-05,9.8,12.7,11.591667,9.0,13.5,10.316667,3


In [246]:
names = ["airtemp_min",
         "airtemp_max",
         "airtemp_mean",
         "windspeed_min",
         "windspeed_max",
         "windspeed_mean"]

_n = {c : n for c, n in zip(w_df.columns[-7:], names)}
w_df = w_df.rename(columns=_n)

In [247]:
w_df.head()

,day,airtemp_min,airtemp_max,airtemp_mean,windspeed_min,windspeed_max,windspeed_mean,site_id
0,2016-02-01,9.1,14.2,11.782609,2.9,10.4,6.343478,3
1,2016-02-02,6.0,12.0,9.700000,2.7,13.0,8.266667,3
2,2016-02-03,8.1,13.9,12.004167,5.8,13.1,9.887500,3
3,2016-02-04,2.4,10.4,6.925000,3.1,10.7,6.433333,3
4,2016-02-05,9.8,12.7,11.591667,9.0,13.5,10.316667,3


**DATA MERGING**

In [248]:
full_df = train.merge(metadata, how="left", on=["building_id"])
full_df = full_df.merge(w_df, how="left", on=["day", "site_id"])

In [249]:
target = "meter_reading"
meter_dict = {595:0, 207:1, 649:2, 413:3}
full_df["meter"].replace(meter_dict, inplace=True)
X = full_df.drop(columns=[target]).loc[full_df[target].isna()==False]
Y = full_df[target].loc[full_df[target].isna()==False]

In [250]:
test = test.merge(metadata, on='building_id', how='left')
test = test.merge(w_df, how="left", on=["day", "site_id"])

**PREDICTIONS**

In [145]:
X_train, X_valid, Y_train, Y_valid = split_time_data(full_df)

model = RandomForestRegressor(n_estimators=15, random_state=100).fit(X_train, Y_train)
score = model.score(X_valid, Y_valid)
print(score)

0.6067091366524604


In [147]:
df7 = make_shifts(full_df, 8)
X_train, X_valid, Y_train, Y_valid = split_time_data(df7)

model = RandomForestRegressor(n_estimators=15, random_state=100).fit(X_train, Y_train)
score = model.score(X_valid, Y_valid)

print(score)

0.6098343898396245


**PREDICTIONS**

In [148]:
predictions = []

for row in range(test.shape[0]):
    lags = pd.DataFrame(return_lags(test.iloc[row]["day"], test.iloc[row]["building_id"], test.iloc[row]["meter"], shifts=8), index=[0]).fillna(0)
    row_to_predict = pd.concat([test.drop(columns=["day"]).iloc[row: row + 1].reset_index(drop=True), lags.reset_index(drop=True)], ignore_index=False, axis="columns")
    predictions.append(model.predict(row_to_predict)[0])
    total_df.loc[row, "meter_reading"] = predictions[-1]
    if row % 100 == 0:
        print("{}/{} complete".format(row, test.shape[0]))

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- airtemp_max
- airtemp_mean
- airtemp_min
- day_number
- electricity
- ...


In [ ]:
def create_prediction_file(results:list | np.ndarray, results_dir="./"):
    """
    Function that formats predictions and writes them to a .csv file ready for submission.

    :param: results         results to write to the file, list | array
    :param: results_dir     directory to write the results file to, str, default current working directory
                            make sure the directory exists before writing the files there.

    """
    csv_fname = "results_{}.csv".format(datetime.now().strftime('%b%d_%H-%M-%S'))
    with open(os.path.join(results_dir, csv_fname), 'w') as f:
        f.write('id,meter_reading\n')
        for i, value in enumerate(results):
            f.write(str(i) + ',' + str(max(0, value)) + '\n')
    return True

In [ ]:
create_prediction_file(predictions)